In [1]:
import pandas as pd
import numpy as np
import itertools
from pandas.api.types import CategoricalDtype

In [2]:
key_mapping={
    'C':0,
    'D':2,
    'E':4,
    'F':5,
    'G':7,
    'A':9,
    'B':11
}

In [3]:
#1 by 1 key to number
def key2num(key):  
  key=key.upper()
  num=key_mapping[key[0]]
  modifier=len(key)
  if modifier==1:
    return num
  elif key[1]=='#':
    return (num+(modifier-1))%12
  elif key[1]=='B':
    return (num-(modifier-1))%12
  elif key[1]=='X':
    return (num+(modifier-1)*2)%12

In [4]:
# key_list to number_list
def keys2num(keys):
  if keys[-1]=='-':
    return sorted([key2num(key) for key in keys[:-1]])
  else:
    return sorted([key2num(key) for key in keys])

In [5]:
df=pd.read_csv('/content/chordToNoteResult.csv')


In [6]:
df['Chord'].unique()

array(['I', 'I7', 'bII', 'II', 'II7', 'III', 'III7', 'IV', 'IV7', 'V',
       'V7', 'bVI', 'GerVI', 'FreVI', 'ItaVI', 'VI', 'VI7', 'VII', 'VII7',
       'DimVII7', 'I+', 'IV+', 'V+', 'V+7', 'DimVII'], dtype=object)

In [7]:
#custom chord likelihood
chord_likelihood_order = CategoricalDtype(
    ['I', 'II', 'III', 'IV', 'V', 'VI', 'VII', 'I7', 'II7', 'III7', 'IV7',
       'V7', 'VI7', 'VII7', 'DimVII','DimVII7', 'bII', 'bVI',
        'I+', 'IV+', 'V+', 'V+7', 'GerVI', 'FreVI', 'ItaVI'],
    ordered=True
)
df['Chord'] = df['Chord'].astype(chord_likelihood_order)

In [8]:
df_copy=df.copy()
df_copy=df_copy.drop(['Unnamed: 0','Key','Chord'],axis=1)

In [9]:
#array: store every chord's key in the same order wrt the csv file
known_key=list(df_copy.to_numpy())
for idx in range(len(known_key)):
  known_key[idx]= sorted(keys2num(known_key[idx]))

In [10]:
#Return whether given arr is a subset of a array in list_arrays
def subset_in_list(arr, list_arrays):
    for array in list_arrays:
      for each in arr:
        if not (each in array):
          break
      else:
        return True
    return False

In [11]:
#all combinations of 4 key notes
combination=np.array(list(itertools.chain.from_iterable(itertools.combinations(np.arange(12),n) for n in range(5))))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [12]:
#array: possible chord_idx in the same order wrt combination
key_chord_idx=[]
for combo in combination:
  chord=[]
  for chord_idx in range(len(known_key)):
    for note in combo:
      if not(note in known_key[chord_idx]):
        break
    else:
      chord.append(chord_idx)
  key_chord_idx.append(chord)

In [13]:
#key --> chord_idx mapping : 
key_chord_mapping={}
for idx in range(len(combination)):
  key_chord_mapping[str(combination[idx])]=key_chord_idx[idx]

In [14]:
#key --> chord_name mapping :  with sorting by chord_likelihood
key_chord_name_mapping={}
for key in key_chord_mapping:#[''.join(df.loc[idx,['Key','Chord']].sort_values('Chord').tolist()) for idx in key_chord_mapping[key]]
  chord_list=df.loc[key_chord_mapping[key],['Key','Chord']].sort_values('Chord')
  key_chord_name_mapping[key]=(chord_list['Key'].astype(str)+chord_list['Chord'].astype(str)).tolist()

In [15]:
import pickle

with open('key_chord_mapping.pickle', 'wb') as handle:
    pickle.dump(key_chord_mapping, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('(sorted by chord_likelihood)key_chord_name_mapping.pickle', 'wb') as handle:
    pickle.dump(key_chord_name_mapping, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [16]:
def keys2chords(keys,threshold=3):
  result=[]
  keys=keys2num(keys)
  keys=list(dict.fromkeys(keys)) #remove duplicates
  for i in range(4,threshold-1,-1):
    for each in itertools.combinations(keys,i):
      result.extend(key_chord_name_mapping[str(each)])
  return result

In [17]:
temp=input()
word=[]
while temp:
  word.append(temp)
  temp=input()

c
e
g



In [18]:
#TEST
import time
answer=''
iterations=100000
start = time.time()
for i in range(iterations):
  answer=keys2chords(word,3)
end = time.time()
print((end - start)/iterations)

4.566118717193603e-06


In [19]:
print(answer)


['CMajorI', 'aMinorIII', 'GMajorIV', 'FMajorV', 'eMinorVI', 'dMinorVII', 'CMajorI7', 'GMajorII7', 'FMajorIII7', 'GMajorIV7', 'FMajorV7', 'CMajorVI7', 'BMajorbII', 'bMinorbII', 'EMajorbVI', 'cMinorI+', 'gMinorIV+', 'fMinorV+', 'fMinorV+7', 'EMajorGerVI', 'eMinorGerVI']
